In [315]:
# Starting paths for dynamically getting all paths.
# We can fix the first two vertex of the path, because if we picked a different 
# position for the second vertex it would have rotational symmetry to the path starting with (0,0),(1,0).
# This will reduce the number of paths to check.
L1 = [[[{(0,0)},set(),(0,0)]],[[{(0,0),(1,0)},set(),(0,0),(1,0)]]]
E1 = [set(frozenset()),set(frozenset())]

# Returns one of the four possible coordinates for the next vertex on the path, num selects between them.
def NextTuple(path, num): 
    return [(path[-1][0]+1, path[-1][1]), (path[-1][0], path[-1][1]+1), (path[-1][0]-1, path[-1][1]), (path[-1][0], path[-1][1]-1)][num]

def NewEdges(path,E):
    edges = set()
    newEdgeSet = set()
    if (path[-1][0]+1, path[-1][1]) in path[0] and (path[-1][0]+1, path[-1][1]) != path[-2]: 
        edges.add((path.index((path[-1][0]+1, path[-1][1]))-1, len(path)-2)) 
    if (path[-1][0], path[-1][1]+1) in path[0] and (path[-1][0], path[-1][1]+1) != path[-2]: 
        edges.add((path.index((path[-1][0], path[-1][1]+1))-1, len(path)-2)) 
    if (path[-1][0]-1, path[-1][1]) in path[0] and (path[-1][0]-1, path[-1][1]) != path[-2]: 
        edges.add((path.index((path[-1][0]-1, path[-1][1]))-1, len(path)-2)) 
    if (path[-1][0], path[-1][1]-1) in path[0] and (path[-1][0], path[-1][1]-1) != path[-2]: 
        edges.add((path.index((path[-1][0], path[-1][1]-1))-1, len(path)-2)) 
    newEdgeSet = path[1] | edges
    if frozenset(newEdgeSet) not in E[len(path)-3]: 
        E[len(path)-3].add(frozenset(newEdgeSet))
    elif newEdgeSet != set():
        newEdgeSet = set('@') # @ symbol is indicating that it is a duplicate 
    return [path[0] | {path[-1]}] + [newEdgeSet] + path[2:]

def TotalNumContacts(path, protein):
    contacts = sum([int(protein[i-1]) * int(protein[i]) for i in range(1, len(protein))])
    contacts += sum([int(protein[edge[0]-1]) * int(protein[edge[1]-1]) for edge in path[1]])
    #print(protein, contacts)
    return contacts

# Dynamically generates a list of all possible paths.
# Currently the filter is filtering out too many paths.
# If the filter is commented out it will work correctly.
# I am working on figuring out a better filter.
def GetPaths(L, E, N): 
    while len(L) <= N: 
        L.append([L[-1][j]+[NextTuple(L[-1][j], i)] for j in range(len(L[-1])) for i in range(4) if NextTuple(L[-1][j], i) not in L[-1][j][0]])
        E.append(set(frozenset()))
        L[-1] = [NewEdges(path,E) for path in L[-1]]
        L[-1] = list(filter(lambda x: "@" not in x[1], L[-1]))

# Simple way to display a path.        
def DisplayPath(path):  
    MaxX = max(path[0], key = lambda x: x[0])[0]
    MaxY = max(path[0], key = lambda x: x[1])[1]
    MinX = min(path[0], key = lambda x: x[0])[0]
    MinY = min(path[0], key = lambda x: x[1])[1]
    
    display = [ " ".join([str(path.index((X,Y))) if (X,Y) in path[0] else "-" for X in range(MinX, MaxX+1)]) for Y in range(MinY, MaxY+1)]
    
    for item in display: print(item)

In [316]:
%%time

N = 9

GetPaths(L1,E1,N-1)
print(len(L1[N-1]))


#for edgeset in E1: print(edgeset)
# All leading and trailing 0's can be removed from the protein,
# and then you can think of it as a protein of a shorter length. Reducing the amount of computation.
proteins = [bin(i)[2:].zfill(N) for i in range(2**N)]

average = sum([max([TotalNumContacts(path,protein) for path in L1[len(protein)-1]]) for protein in proteins])/2**N
#maxcontactlist =[max([TotalNumContacts(path,protein) for path in L1[len(protein)-1]]) for protein in proteins]

print("Answer for N =",N, "is", average)
#print(maxcontactlist)
#for i in range(N**2):
#    print(proteins[i],maxcontactlist[i])
#for path in L1[N-1]:
#    print()
#    DisplayPath(path)

596
Answer for N = 9 is 3.72265625
Wall time: 1.95 s
